In [ ]:
# set up our local code path
import sys
sys.path.append('/home/serenity/dev/shadows/serenity/serenity-mdrecorder/src')

# standard imports
import pandas as pd
import plotly.graph_objects as go

from cloudwall.serenity.mdrecorder.tickstore import LocalTickstore
from datetime import datetime as dt
from pathlib import Path
from plotly.subplots import make_subplots

# set up Cufflinks
import cufflinks as cf
cf.set_config_file(theme='ggplot',sharing='public',offline=True)

In [ ]:
# Connect to Behemoth
tickstore = LocalTickstore(Path('/behemoth/db/COINBASE_PRO_ONE_MIN_BINS'), 'time')


In [ ]:
df = tickstore.select('BTC-USD', start=dt(2015, 7, 20), end=dt(2019, 12, 31))

In [ ]:
df['volume'].max()

In [ ]:
hist_1d = df.resample('1D').agg({'open': 'first', 
                                 'high': 'max', 
                                 'low': 'min', 
                                 'close': 'last',
                                 'volume': 'sum'
                                 })
hist_1d['close'].iplot()

In [ ]:
hist_1h = df.resample('1H').agg({'open': 'first', 
                                 'high': 'max', 
                                 'low': 'min', 
                                 'close': 'last',
                                 'volume': 'sum'
                                 })

hist_1h['hour_of_day'] = hist_1h.index.hour
hist_1h['month_of_year'] = hist_1h.index.month
hist_1h['pct_change'] = hist_1h['close'].pct_change(1)
hist_1h['volatility'] = hist_1h['pct_change'].rolling('1D').std()*(252**0.5)

hist_1h.head()

In [ ]:
hist_1h['volume'].iplot(kind='histogram')
hist_1h['pct_change'].iplot(kind='histogram')
hist_1h.iplot(kind='bubble', size='volume', x='month_of_year', y='hour_of_day')

In [ ]:
hist_1h.iplot(kind='heatmap', colorscale='Blues', x='hour_of_day', y='pct_change', z='volatility')

In [ ]:
hist_1h['volatility'].iplot()
hist_1h.iplot(kind='bubble', size='volume', x='hour_of_day', y='volatility')

In [ ]:
trades_tickstore = LocalTickstore(Path('/behemoth/db/COINBASE_PRO_TRADES'), 'time')
cbp_trades = trades_tickstore.select('BTC-USD', start=dt(2019, 11, 26), end=dt(2019, 11, 30))


In [ ]:
cbp_trades.head()

In [ ]:
def plot_trades(trades, title: str):
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(
        go.Scatter(x=trades.index, y=trades['price'], name="price ($)"),
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(x=trades.index, y=trades['size'], name="size (BTC)"),
        secondary_y=True,
    )

    # Add figure title
    fig.update_layout(
        title_text=title
    )

    fig.show()
    
def plot_ohlcv(ohlcv, title: str):
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(
        go.Ohlc(x=ohlcv.index, open=ohlcv['open'], low=ohlcv['low'], high=ohlcv['high'], close=ohlcv['close'], name="price ($)"),
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(x=ohlcv.index, y=ohlcv['size'], name="size (BTC)"),
        secondary_y=True,
    )

    # Add figure title
    fig.update_layout(
        title_text=title
    )

    fig.show()
    
    
def bin_trades(trades, bin_size='1min'):
    price = trades['price'].resample(bin_size).ohlc().bfill()
    vol = trades['size'].resample(bin_size).sum().bfill()
    vol.columns = pd.MultiIndex.from_tuples([('size', 'sum')])
    return pd.concat([price, vol], axis=1)


In [ ]:
binance_tickstore = LocalTickstore(Path('/behemoth/db/BINANCE_TRADES'), 'time')
binance_trades = binance_tickstore.select('BTC-USDT', start=dt(2019, 11, 26), end=dt(2019, 11, 27))

In [ ]:
binance_trades.head()

In [ ]:
cbp_1min_bins = bin_trades(cbp_trades)
binance_1min_bins = bin_trades(binance_trades)

plot_ohlcv(cbp_1min_bins, "BTC-USD price & volumes (Coinbase Pro, 1 min bins)")
plot_ohlcv(binance_1min_bins, "BTC-USDT price & volumes (Binance, 1 min bins)")

In [ ]:
cbp_bins = bin_trades(cbp_trades, '10ms')
binance_bins = bin_trades(binance_trades, '10ms')

corr = cbp_bins['close'].corr(binance_bins['close'])

merged_prices = pd.merge_asof(cbp_bins[['close']], binance_bins[['close']], on='time')

print(corr)

# Create figure with secondary y-axis
fig = go.Figure()

# Add traces
fig.add_trace(
    go.Scatter(x=merged_prices.index, y=merged_prices['close_x'], name="Coinbase Pro")
)

fig.add_trace(
    go.Scatter(x=merged_prices.index, y=merged_prices['close_y'], name="Binance")
)

# Add figure title
fig.update_layout(
    title_text="Tracing Binance & Coinbase Pro prices"
)

fig.show()

In [ ]:
# https://towardsdatascience.com/four-ways-to-quantify-synchrony-between-time-series-data-b99136c4a9c9
import scipy.stats as stats

r, p = stats.pearsonr(merged_prices['close_x'], merged_prices['close_y'])
print(r, p)